<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/tasca1/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import cv2
import random
import numpy as np
import os


from google.colab import files
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from skimage import io
import kagglehub
import pandas as pd
from glob import glob


from torchvision import transforms,models
from torch import nn
from torch.utils.data import Dataset

# !rm -rf /content/sample_data/*

# !kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

# !unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/

!rm -rf /content/sample_data/*

!kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

!unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/

In [ ]:
metadates = pd.read_csv('/content/sample_data/HAM10000_metadata.csv')
metadates = metadates.sort_values(by='image_id')
metadates.head()
print(metadates['dx'].value_counts())
print()
print(metadates['dx'].value_counts() / sum(metadates['dx'].value_counts()))

dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64

dx
nv       0.669496
mel      0.111133
bkl      0.109735
bcc      0.051323
akiec    0.032651
vasc     0.014179
df       0.011483
Name: count, dtype: float64


nv: melanocytic nevi

vasc: vascular lesions

mel:melanoma

df: dermatofibroma

bkl: benign keratosis-like lesions

bcc: basal cell carcinoma

akiec: Actinic keratoses and intraepithelial carcinoma / Bowen's disease

In [ ]:
class Formes(Dataset):
  #Classe on gestionarem les imatges dels fitxers
  dict_illnesses = {0 : 'nv', 1 : 'mel', 2 : 'bkl', 3 : 'bcc', 4 : 'akiec', 5 : 'vasc', 6 : 'df'}

  def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform
        #Per defecte pens que el color pot extreure característiques importants, per tant en primer lloc
        #entrenarem les imatges de color
        self.greyscale = False

  def __len__(self):
        return self.len

  def __addlabel__(self,label):
    self.labels = np.append(self.labels,label)

  def __addPath__(self,path):
    self.paths = np.append(self.paths,path)

  def __getdist__(self):
    return pd.Series(self.labels).value_counts()

  def __getlabels__(self):
    classes = [[],[],[],[],[],[],[]]
    for i in range(len(self.labels)):
      classes[self.labels[i]].append(i) #afegim l'index
    return classes

  def __redistribute__(self,percentages):
    #percentages: [15,15,10,10,5,5] percentatges que volem pujar de la resta de classes llevat de nv
    threshold = 0.005  #percentatge de marge que deixam a la redistribució
    Ntarget = self.len
    classes = self.__getlabels__() #indexos de cada clase
    afegir = np.array([0,0,0,0,0,0],dtype=np.int64) # de nv mai haurem d'afegir

    nmel = len(classes[1]) #nombre inicial de cada clase
    nbkl = len(classes[2])
    nbcc = len(classes[3])
    nakiec = len(classes[4])
    nvasc = len(classes[5])
    ndf = len(classes[6])

    while True:

      suma_actual = afegir.sum()

      operacio = percentages[0]*Ntarget - nmel
      afegir[0] +=  operacio if operacio > 0 else 0
      nmel += operacio if operacio > 0 else 0

      operacio = percentages[1]*Ntarget - nbkl
      afegir[1] += operacio if operacio > 0 else 0
      nbkl += operacio if operacio > 0 else 0

      operacio = percentages[2]*Ntarget - nbcc
      afegir[2] += operacio if operacio > 0 else 0
      nbcc += operacio if operacio > 0 else 0

      operacio = percentages[3]*Ntarget - nakiec
      afegir[3] += operacio if operacio > 0 else 0
      nakiec += operacio if operacio > 0 else 0

      operacio = percentages[4]*Ntarget - nvasc
      afegir[4] += operacio if operacio > 0 else 0
      nvasc += operacio if operacio > 0 else 0

      operacio = percentages[5]*Ntarget - ndf
      afegir[5] += operacio if operacio > 0 else 0
      ndf += operacio if operacio > 0 else 0

      if (afegir.sum()-suma_actual) < Ntarget*threshold:
        break

      Ntarget += (afegir.sum()-suma_actual)

    #Quedaria afegir a les imatges les còpies
    for i in range(len(afegir)):
      for j in range(afegir[i]):

          self.__addPath__(self.paths[classes[i+1][random.randint(0, len(classes[i+1]) - 1)]])
          self.__addlabel__(i+1)

    self.len = len(self.labels)

  def __setgreyscale__(self,mode):
    self.greyscale = mode

  def __getitem__(self, index):
      path = self.paths[index]
      label = self.labels[index]

      image = cv2.imread(path, cv2.IMREAD_GRAYSCALE if self.greyscale else cv2.IMREAD_COLOR)  # Depén de vosaltres
      image = self.transform(image)

      return image, label

In [ ]:
def calcula_mitjana_desviacio(img_files_path):
  mitjana = np.array([0.0, 0.0, 0.0], dtype=np.float64)
  desviacio = np.array([0.0, 0.0, 0.0], dtype=np.float64)

  x,y = 224,224
  pixels_totals_canal = len(img_files_path) * x * y

  for i in range(len(img_files_path)):
    imatge = cv2.imread(img_files_path[i])
    imatge = cv2.resize(imatge,(x,y))

    canal_b, canal_g, canal_r = cv2.split(imatge)

    mitjana += np.array([canal_r.sum(), canal_g.sum(), canal_b.sum()])
    #desviacio += np.array([(canal_r**2).sum(), (canal_g**2).sum(), (canal_b**2).sum()])

  mitjana = mitjana / pixels_totals_canal

  for i in range(len(img_files_path)):
    imatge = cv2.imread(img_files_path[i])
    imatge = cv2.resize(imatge,(x,y))

    canal_b, canal_g, canal_r = cv2.split(imatge)
    desviacio += np.array([((canal_r-mitjana[0])**2).sum(), ((canal_g-mitjana[1])**2).sum(), ((canal_b-mitjana[2])**2).sum()])


  desviacio = np.sqrt(desviacio / pixels_totals_canal)

  return mitjana,desviacio

In [ ]:
TRAINING = 0.64
VALIDATION = 0.16
TESTING = 0.20

transform = transforms.Compose([
    transforms.ToTensor(),
])

illnes_dictionary = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses and intraepithelial carcinoma / Bowens disease',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

img_files_1 = sorted(glob('/content/sample_data/HAM10000_images_part_1/*'))
img_files_2 = sorted(glob('/content/sample_data/HAM10000_images_part_2/*'))
img_files = img_files_1 + img_files_2

img_files = np.array(img_files)

#mitjana,desviacio = calcula_mitjana_desviacio(img_files)
#S'ha descobert amb aquesta funció que la mitjana = [35.92275236 38.90347617 43.33101831]
#I desviació= [35.92275236 38.90347617 43.33101831]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = [35.92275236, 38.90347617, 43.33101831], std=[35.92275236, 38.90347617, 43.33101831])
])

imgid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in img_files}

metadates['path'] = metadates['image_id'].map(imgid_path_dict.get)
metadates['illness'] = metadates['dx'].map(illnes_dictionary.get)
metadates['illness_code'] = metadates['dx'].map({'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3, 'akiec': 4, 'vasc': 5, 'df': 6})

#Aquest illness_code s'utilitzarà com a label de la enfermetat

img_number = len(img_files)


X = metadates.drop('illness_code',axis= 1)
y = metadates['illness_code']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TESTING, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VALIDATION/(TRAINING+VALIDATION), random_state=42, stratify=y_train)

#Una bona idea seria aplicar data augmentation al conjunt d'entrenament ja que les classes estan molt desbalancejades
#Una altra bona idea seria emplear una funció de pèrdua que tengui en compte les classes desbalancejades.

#Antes de guardar els datasets en classes Formes, hauriem de caluclar la mitjana i desviació típica de les imatges
train_data = Formes(X_train['path'].to_numpy(),y_train.to_numpy(),transform)
test_data = Formes(X_test['path'].to_numpy(),y_test.to_numpy(),transform)
validation_data = Formes(X_val['path'].to_numpy(),y_val.to_numpy(),transform)


print(train_data.__getdist__())
train_data.__redistribute__([0.12,0.12,0.06,0.04,0.02,0.02])
print(train_data.__getdist__())
print(train_data.len)





0    4291
1     712
2     703
3     329
4     209
5      91
6      74
Name: count, dtype: int64
0    4291
2     823
1     823
3     410
4     273
6     136
5     136
Name: count, dtype: int64
6892


In [ ]:
# labels = numpy.array(metadates['dx'
# labels = metadates['dx'].to_numpy()
# idx_list = list(range(img_number))
# random.shuffle(idx_list)
# training_imgs = img_files[idx_list[:int(img_number*TRAINING)]]
# training_labels = labels[idx_list[:int(img_number*TRAINING)]]
# train_data = Formes(training_imgs,training_labels,transform)
# validation_imgs = img_files[idx_list[int(img_number*TRAINING):int(img_number*(TRAINING+VALIDATION))]]
# validation_labels = labels[idx_list[int(img_number*TRAINING):int(img_number*(TRAINING+VALIDATION))]]
# validation_data = Formes(validation_imgs,validation_labels,transform)
# testing_imgs = img_files[idx_list[int(img_number*(TRAINING+VALIDATION)):]]
# testing_labels = labels[idx_list[int(img_number*(TRAINING+VALIDATION)):]]
# testing_data = Formes(testing_imgs,testing_labels,transform)

##ENTRENAMENT

In [ ]:
alexnetbinary = models.alexnet(pretrained=False)
alexnetmulticlass = models.alexnet(pretrained=False)

alexnetbinary.classifier = nn.Sequential(
    torch.nn.Linear(9216, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 512),
    nn.ReLU(),
    torch.nn.Linear(512, 1),
    nn.Sigmoid()
)
alexnetmulticlass.classifier = nn.Sequential(
    torch.nn.Linear(9216, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 512),
    nn.ReLU(),
    torch.nn.Linear(512, 7),  # Ja que tenim 7 classes.
    nn.Softmax(dim=1)
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
